# Azure SQL Database
In this notebook we will access Azure SQL database

In [1]:
# install all the azure identity modules
%pip install azure.identity
%pip install pandas
# install all the azure sql database modules
%pip install pyodbc

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.



In [2]:
from azure.identity import AzureCliCredential, ChainedTokenCredential, DefaultAzureCredential, ManagedIdentityCredential
import struct
import pyodbc
import pandas as pd 

class DatabaseConnection():
    def __init__(self, configuration : dict):
        server = configuration.get('server')
        if (not str(server).endswith('.database.windows.net')) :
            server = server + '.database.windows.net';
        self.sever = server;
        self.database = configuration.get('database');

        self.server = 'eppmreportingdev.database.windows.net';
        self.database = 'DCEPDev';
        query = 'SELECT TOP 10 * from [MonteCarlo].[SimulationResult];'
        credential = ChainedTokenCredential(ManagedIdentityCredential(), 
                                            AzureCliCredential(), 
                                            DefaultAzureCredential());
        databaseToken = credential.get_token('https://database.windows.net/');
        print(databaseToken);
        tokenb = bytes(databaseToken[0], "UTF-8");
        exptoken = b'';
        for i in tokenb:
            exptoken += bytes({i});
            exptoken += bytes(1);
        tokenstruct = struct.pack("=i", len(exptoken)) + exptoken;
        connString = "Driver={ODBC Driver 17 for SQL Server};SERVER="+self.sever+";DATABASE="+self.database+"";
        SQL_COPT_SS_ACCESS_TOKEN = 1256; 
        self.connection = pyodbc.connect(connString, attrs_before = {SQL_COPT_SS_ACCESS_TOKEN:tokenstruct});


    # write a database to database, mode = append or replace
    def insert(self, df : pd.DataFrame, table :str):
        """
        Execute a SQL write.
        @param df: Data Frame.        
        @param table: Table Name.
        """
        statement = '''
        INSERT INTO {table}
        (
            {columns}
        )
        VALUES({place_holders})
        '''
        # add protection on column name
        columns = df.columns.tolist();
        for i in range(len(columns)):
            columns[i] = '[' + columns[i] + ']';
        statement = statement.format(table = table, 
                                     columns = (',').join(columns), 
                                     place_holders = (',').join(['?'] * len(df.columns)));
        print(statement);
        # Create a Pandas dataframe from the results
        cursor = self.connection.cursor();
        cursor.fast_executemany = True;
        cursor.executemany(statement, df.values.tolist());
        cursor.commit();
        cursor.close();

    # execute sql command
    def execute(self, statement : str):
        """
        Execute a SQL command.
        @param query: query.        
        """
        # Create a Pandas dataframe from the results
        cursor = self.connection.cursor();
        cursor.execute(statement);
        cursor.commit();
        cursor.close();


    def query(self, query: str) -> pd.DataFrame:
        """
        Execute a SQL query.
        @param query: The query.
        @returns: A Pandas DataFrame.
        """
        # Create a Pandas dataframe from the results
        
        df = pd.read_sql(query, self.connection);

        # Print the dataframe
        print(df);
        return df

    def close(self):
        """
        Close the Database client.
        """
        self.connection.close();


databaseConn = DatabaseConnection({'server' : 'eppmreportingdev.database.windows.net', 'database' :'DCEPDev'});
df = databaseConn.query('SELECT * FROM [CSPW].[GPURetrofit_Signal_Fact]');
databaseConn.close();
print(df);
    


C:\Users\huanminw\AppData\Local\Temp\ipykernel_23324\2133893049.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


AccessToken(token='eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsIng1dCI6IlhSdmtvOFA3QTNVYVdTblU3Yk05blQwTWpoQSIsImtpZCI6IlhSdmtvOFA3QTNVYVdTblU3Yk05blQwTWpoQSJ9.eyJhdWQiOiJodHRwczovL2RhdGFiYXNlLndpbmRvd3MubmV0LyIsImlzcyI6Imh0dHBzOi8vc3RzLndpbmRvd3MubmV0LzcyZjk4OGJmLTg2ZjEtNDFhZi05MWFiLTJkN2NkMDExZGI0Ny8iLCJpYXQiOjE3MTA0MzY4OTAsIm5iZiI6MTcxMDQzNjg5MCwiZXhwIjoxNzEwNDQxNTUxLCJfY2xhaW1fbmFtZXMiOnsiZ3JvdXBzIjoic3JjMSJ9LCJfY2xhaW1fc291cmNlcyI6eyJzcmMxIjp7ImVuZHBvaW50IjoiaHR0cHM6Ly9ncmFwaC53aW5kb3dzLm5ldC83MmY5ODhiZi04NmYxLTQxYWYtOTFhYi0yZDdjZDAxMWRiNDcvdXNlcnMvOTJlYzgxNWYtZGM0Yy00ZWZmLWFmNGYtMzRjMjc5ZGI4YmIzL2dldE1lbWJlck9iamVjdHMifX0sImFjciI6IjEiLCJhaW8iOiJBYlFBUy84V0FBQUFGWDFqM0J0bitCUlRwQkJoWVhjZHMya3hPQjJzdUFZS3AxMnhDemNLanRudThydDd6cDZOMFJ0MTVFMFZDNThLSWFJMTJZSVRocG1RcTM1elphTmxrcEtvUkRQVzBsT212b2tLS2IzNkFDL3FQRExkTWZpNmEwQzRiWks3VHdsWU9IOE1WQjRnSW1ubUNiQlRrS3JHZjJ3TXZFWFhCd3QwYWNWTGpvVHg5enhpYzR2RC9HMEREOFprTXRaN0ZmaEplWUp2SzJHNEdIbWRwWk5DSmFDL1ZPWVhscmhXQ1dxbW9OM2VDcUpYbWNJPSIsImFtciI6WyJyc2Ei

C:\Users\huanminw\AppData\Local\Temp\ipykernel_23324\2133893049.py:83: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, self.connection);
